In [16]:
import torch
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random 
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
import string
from nltk.corpus import stopwords
plt.ion()

In [45]:
%time
def clean_data(reviews):
    reviews.text = reviews.text.apply(lambda x: x.lower())
    
    def remove_punctuation(s):
    #s = s.split()
    
        s = [ch for ch in s if ch not in string.punctuation]

        
        s = "".join(s)
        
        s = [c for c in s if c == " " or c.isalnum()]
        
        s = "".join(s)
        
        return s
    
    
    def remove_stopwords(s):
        stop_words = set(stopwords.words("english"))

        s = s.split(" ")

        s = [word for word in s if word not in stop_words]
        
        
        s = " ".join(s)

        
        
        
        
        return s
    
    
    reviews.text = reviews.text.apply(lambda x:remove_punctuation(x))
    reviews.text = reviews.text.apply(lambda x:remove_stopwords(x))
    
    return reviews

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs


In [28]:
%%time
reviews = pd.read_json("/home/bh_parijat/data/yelp/review.json",lines=True)

CPU times: user 1min 27s, sys: 25.7 s, total: 1min 53s
Wall time: 1min 52s


In [41]:
%%time
reviews.head()

CPU times: user 428 µs, sys: 41 µs, total: 469 µs
Wall time: 409 µs


,review_id,user_id,business_id,stars,useful,funny,cool,text,date,text_length
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1,6,1,0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36,204
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5,0,0,0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33,1561
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5,3,0,0,I have to say that this office really has it t...,2016-11-09 20:09:03,615
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5,0,0,0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38,407
4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1,7,0,0,Today was my second out of three sessions I ha...,2018-01-30 23:07:38,3509


In [49]:
%%time
reviews_cleaned = reviews.copy()
reviews_cleaned = clean_data(reviews_cleaned)

CPU times: user 26min 23s, sys: 54.9 s, total: 27min 18s
Wall time: 27min 18s


In [89]:
filter_null_row_cond = reviews_cleaned.text == "null"
reviews_cleaned = reviews_cleaned[~filter_null_row_cond]

In [90]:
reviews['text_length'] = reviews.text.apply(lambda x:len(x))

In [91]:
reviews_cleaned.columns

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date', 'text_length', 'label'],
      dtype='object')

In [92]:
reviews_cleaned.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,text_length,label
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1,6,1,0,total bill horrible service 8gs crooks actuall...,2013-05-07 04:34:36,129,0
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5,0,0,0,adore travis hard rocks new kelly cardenas sal...,2017-01-14 21:30:33,1064,1
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5,3,0,0,say office really together organized friendly ...,2016-11-09 20:09:03,369,1
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5,0,0,0,went lunch steak sandwich delicious caesar sal...,2018-01-09 20:56:38,279,1
4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1,7,0,0,today second three sessions paid although firs...,2018-01-30 23:07:38,2021,0


In [93]:
reviews.shape,reviews_cleaned.shape

((6685900, 10), (6685899, 11))

In [94]:
reviews_cleaned['label']= reviews.stars.apply(lambda x: 1 if x>=3 else 0)

In [95]:
%%time
reviews_cleaned.to_csv("model/master_reviews.csv",index=False)

CPU times: user 1min 56s, sys: 3.23 s, total: 1min 59s
Wall time: 2min 3s
